This box will set up the name of the slice and set the expected resources<br>

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

exp_requires = {'core': 10*2, 'nic': 20}
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_requires['core']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_requires['nic']) ):
        break


import os
slice_name="IP_Test_" + os.getenv('NB_USER')

try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
except:
    print("You don't have a slice named %s yet." % slice_name)
    slice = fablib.new_slice(name=slice_name)

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
Bastion Username,maross6_0000108606
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


You already have a slice by this name!


<br><br>Next we name and set the networks and interfaces<br>

In [44]:
# this cell sets up the hosts and router
node_names = ["c1","c2","c3","p1","p2","p3", "r1", "r2", "r3","r4"]
for n in node_names:
    slice.add_node(name=n, site=site_name, cores=8, ram=32, disk=10, image='default_ubuntu_22')
    
# this cell sets up the network links
nets = [
    {"name": "net1",   "nodes": ["r1","c1","c2"]},
    {"name": "net2",   "nodes": ["r2","p1"]},
    {"name": "net3",   "nodes": ["r3","p2"]},
    {"name": "net4",   "nodes": ["r4","p3","c3"]},
    {"name": "net12",   "nodes": ["r1","r2"]},
    {"name": "net23",   "nodes": ["r2","r3"]},
    {"name": "net34",   "nodes": ["r3","r4"]},
    {"name": "net41",   "nodes": ["r4","r1"]},
]
for n in nets:
    ifaces = [slice.get_node(node).add_component(model="NIC_Basic", name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

<br><br>Request the slice and wait until it has been reserved<br>

In [45]:
slice.submit()


Retry: 13, Time: 1045 sec


ID,e4312ec7-9de1-41a0-81b6-122f552c2486
Name,IP_Test_maross6
Lease Expiration (UTC),2023-11-13 21:16:45 +0000
Lease Start (UTC),2023-11-12 21:16:46 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
1b4f693c-7b34-4340-9126-48eb24d30b89,c1,8,32,10,default_ubuntu_20,qcow2,ncsa-w3.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fed6:c539,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fed6:c539,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
20aaa179-59d7-4510-9b1d-123c3a28bc75,c2,8,32,10,default_ubuntu_20,qcow2,ncsa-w3.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fe20:d197,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe20:d197,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
34b344af-4b80-40ff-bbd5-ef255b8c5d4f,c3,8,32,10,default_ubuntu_20,qcow2,ncsa-w3.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fe9c:c86e,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe9c:c86e,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
f198b624-d9fc-45f7-85f4-047d75f0a618,p1,8,32,10,default_ubuntu_20,qcow2,ncsa-w3.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fea1:c166,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fea1:c166,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
59323066-2390-4c47-acfb-49b2a116277d,p2,8,32,10,default_ubuntu_20,qcow2,ncsa-w3.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:feba:6c4c,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:feba:6c4c,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
a08aa31f-f715-4591-bcaf-f2da17c496c5,p3,8,32,10,default_ubuntu_20,qcow2,ncsa-w3.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:feb7:c555,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:feb7:c555,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
7bc6cd06-123e-41e3-b31d-a09020f207ee,r1,8,32,10,default_ubuntu_20,qcow2,ncsa-w3.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:feb1:2fdb,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:feb1:2fdb,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
af560cbf-ceb1-46bb-bec5-cca902821e11,r2,8,32,10,default_ubuntu_20,qcow2,ncsa-w3.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fe96:82c9,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe96:82c9,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
89a51c76-eb7e-429b-8e11-98178e71eb78,r3,8,32,10,default_ubuntu_20,qcow2,ncsa-w3.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fe5d:acdf,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe5d:acdf,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
a1b368ab-dd28-47ac-a74d-10f140df4893,r4,8,32,10,default_ubuntu_20,qcow2,ncsa-w3.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fea4:b792,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fea4:b792,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
56be1f61-3d90-4481-b521-db141aaeedc2,net1,L2,L2Bridge,NCSA,None,None,Active,
0e957912-6b50-4b73-b5d4-a6e22783059a,net12,L2,L2Bridge,NCSA,None,None,Active,
919f6186-9678-4a86-bdd6-9f16590c1ac5,net2,L2,L2Bridge,NCSA,None,None,Active,
7c1f1ef9-f5d3-47c3-988d-02887178e904,net23,L2,L2Bridge,NCSA,None,None,Active,
e566cf8a-818e-4c3e-9aed-589dddbaecf5,net3,L2,L2Bridge,NCSA,None,None,Active,
0017e320-adca-4ae5-80d5-18a4f99eac3e,net34,L2,L2Bridge,NCSA,None,None,Active,
be6f3071-7a00-4955-aa8b-3091327ea288,net4,L2,L2Bridge,NCSA,None,None,Active,
8a81d412-c170-4d5a-8772-1280c079fe8e,net41,L2,L2Bridge,NCSA,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
c1-net1-p1,p1,c1,net1,100,config,,16:89:1A:F5:6C:32,enp7s0,enp7s0,None,4
c2-net1-p1,p1,c2,net1,100,config,,16:DC:EF:D5:FD:30,enp7s0,enp7s0,None,4
c3-net4-p1,p1,c3,net4,100,config,,1E:B5:FB:92:B6:86,enp7s0,enp7s0,None,4
p1-net2-p1,p1,p1,net2,100,config,,1A:F8:20:58:B8:C1,enp7s0,enp7s0,None,4
p2-net3-p1,p1,p2,net3,100,config,,26:1F:B2:EE:01:D4,enp7s0,enp7s0,None,4
p3-net4-p1,p1,p3,net4,100,config,,26:68:2D:8F:2E:1A,enp7s0,enp7s0,None,4
r1-net12-p1,p1,r1,net12,100,config,,26:8B:F1:7E:F1:3C,enp8s0,enp8s0,None,4
r1-net1-p1,p1,r1,net1,100,config,,26:77:D8:6D:0A:29,enp7s0,enp7s0,None,4
r1-net41-p1,p1,r1,net41,100,config,,26:B6:8C:6D:EE:7B,enp9s0,enp9s0,None,4
r2-net2-p1,p1,r2,net2,100,config,,2A:ED:4A:30:3E:8C,enp7s0,enp7s0,None,4



Time to print interfaces 1073 seconds


'e4312ec7-9de1-41a0-81b6-122f552c2486'

In [46]:
slice.wait_ssh(progress=True)

Waiting for slice . Slice state: StableOK
Waiting for ssh in slice . ssh successful


True

In [47]:
from ipaddress import ip_address, IPv4Address, IPv4Network

if_conf = {
    "c1-net1-p1":  {"addr": "10.0.0.1", "subnet": "10.0.0.0/16"},
    "c2-net1-p1":  {"addr": "10.0.0.2", "subnet": "10.0.0.0/16"},
    "r1-net1-p1":  {"addr": "10.0.1.1", "subnet": "10.0.0.0/16"},
    
    "p1-net2-p1":  {"addr": "20.0.2.1", "subnet": "20.0.0.0/16"},
    "r2-net2-p1":  {"addr": "20.0.1.2", "subnet": "20.0.0.0/16"},
    
    "p2-net3-p1":  {"addr": "30.0.2.2", "subnet": "30.0.0.0/16"},
    "r3-net3-p1":  {"addr": "30.0.1.3", "subnet": "30.0.0.0/16"},
    
    "p3-net4-p1":  {"addr": "40.0.2.3", "subnet": "40.0.0.0/16"},
    "c3-net4-p1":  {"addr": "40.0.0.3", "subnet": "40.0.0.0/16"},
    "r4-net4-p1":  {"addr": "40.0.1.4", "subnet": "40.0.0.0/16"},
    
    "r1-net12-p1":  {"addr": "12.0.1.1", "subnet": "12.0.0.0/16"},
    "r2-net12-p1":  {"addr": "12.0.1.2", "subnet": "12.0.0.0/16"},
    
    "r2-net23-p1":  {"addr": "23.0.1.2", "subnet": "23.0.0.0/16"},
    "r3-net23-p1":  {"addr": "23.0.1.3", "subnet": "23.0.0.0/16"},
    
    "r3-net34-p1":  {"addr": "34.0.1.3", "subnet": "34.0.0.0/16"},
    "r4-net34-p1":  {"addr": "34.0.1.4", "subnet": "34.0.0.0/16"},
    
    "r4-net41-p1":  {"addr": "41.0.1.4", "subnet": "41.0.0.0/16"},
    "r1-net41-p1":  {"addr": "41.0.1.1", "subnet": "41.0.0.0/16"}
}

for iface in slice.get_interfaces():
    if_name = iface.get_name()
    iface.ip_addr_add(addr=if_conf[if_name]['addr'], subnet=IPv4Network(if_conf[if_name]['subnet']))
    
for n in ['r1','r2','r3','r4']:
    slice.get_node(name=n).execute("sudo sysctl -w net.ipv4.ip_forward=1")
    
for iface in slice.get_interfaces():
    iface.ip_link_up()
    

rt_conf = [
    {"name": "r1",  "addr": "20.0.0.0/16", "gw": "12.0.1.2"},
    {"name": "r1",  "addr": "40.0.0.0/16", "gw": "41.0.1.4"},
    {"name": "r1",  "addr": "30.0.0.0/16", "gw": "12.0.1.2"},
    
    {"name": "r2",  "addr": "10.0.0.0/16", "gw": "12.0.1.1"},
    {"name": "r2",  "addr": "30.0.0.0/16", "gw": "23.0.1.3"},
    {"name": "r2",  "addr": "40.0.0.0/16", "gw": "23.0.1.3"},
    
    {"name": "r3",  "addr": "20.0.0.0/16", "gw": "23.0.1.2"},
    {"name": "r3",  "addr": "40.0.0.0/16", "gw": "34.0.1.4"},
    {"name": "r3",  "addr": "10.0.0.0/16", "gw": "34.0.1.4"},
    
    {"name": "r4",  "addr": "10.0.0.0/16", "gw": "41.0.1.1"},
    {"name": "r4",  "addr": "30.0.0.0/16", "gw": "34.0.1.3"},
    {"name": "r4",  "addr": "20.0.0.0/16", "gw": "34.0.1.3"}
    
]

for rt in rt_conf:
    slice.get_node(name=rt['name']).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])
    
    
for n in ["c1","c2","c3","p1","p2","p3", "r1", "r2", "r3","r4"]:
    slice.get_node(name=n).execute("sudo apt update; sudo apt -y install net-tools", quiet=True)
    
for n in ["c1","c2"]:
    slice.get_node(name=n).execute("sudo route add default gw 10.0.1.1 enp7s0", quiet=False)
for n in ["p1"]:
    slice.get_node(name=n).execute("sudo route add default gw 20.0.1.2 enp7s0", quiet=False)
for n in ["p2"]:
    slice.get_node(name=n).execute("sudo route add default gw 30.0.1.3 enp7s0", quiet=False)
for n in ["c3","p3"]:
    slice.get_node(name=n).execute("sudo route add default gw 40.0.1.4 enp7s0", quiet=False)

#for n in ["r1"]:
#    slice.get_node(name=n).execute("sudo route add default gw 40.0.1.4 enp7s0", quiet=False)
#for n in ["r2"]:
#    slice.get_node(name=n).execute("sudo route add default gw 10.0.1.1 enp7s0", quiet=False)
#for n in ["r3"]:
#    slice.get_node(name=n).execute("sudo route add default gw 20.0.1.2 enp8s0", quiet=False)
#for n in ["r4"]:
#    slice.get_node(name=n).execute("sudo route add default gw 30.0.1.3 enp7s0", quiet=False)
    
import pandas as pd
pd.set_option('display.max_colwidth', None)
ssh_str = 'ssh -i ' + slice.get_slice_private_key_file() + \
    ' -J ' + fablib.get_bastion_username() + '@' + fablib.get_bastion_public_addr() + \
    ' -F /home/fabric/work/fabric_config/ssh_config '
slice_info = [{'Name': n.get_name(), 'SSH command': ssh_str + n.get_username() + '@' + str(n.get_management_ip())} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

net.ipv4.ip_forward = 1
net.ipv4.ip_forward = 1
net.ipv4.ip_forward = 1
net.ipv4.ip_forward = 1
 SIOCADDRT: No such device
  SIOCADDRT: No such device
  SIOCADDRT: No such device
  SIOCADDRT: No such device
  SIOCADDRT: No such device
  SIOCADDRT: No such device
  SIOCADDRT: No such device
  SIOCADDRT: Network is unreachable
  SIOCADDRT: Network is unreachable
  SIOCADDRT: No such device
 

,SSH command
Name,
c1,ssh -i /home/fabric/work/fabric_config/slice_key -J maross6_0000108606@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fed6:c539
c2,ssh -i /home/fabric/work/fabric_config/slice_key -J maross6_0000108606@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe20:d197
c3,ssh -i /home/fabric/work/fabric_config/slice_key -J maross6_0000108606@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe9c:c86e
p1,ssh -i /home/fabric/work/fabric_config/slice_key -J maross6_0000108606@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fea1:c166
p2,ssh -i /home/fabric/work/fabric_config/slice_key -J maross6_0000108606@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:feba:6c4c
p3,ssh -i /home/fabric/work/fabric_config/slice_key -J maross6_0000108606@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:feb7:c555
r1,ssh -i /home/fabric/work/fabric_config/slice_key -J maross6_0000108606@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:feb1:2fdb
r2,ssh -i /home/fabric/work/fabric_config/slice_key -J maross6_0000108606@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe96:82c9
r3,ssh -i /home/fabric/work/fabric_config/slice_key -J maross6_0000108606@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe5d:acdf


Test a few connections through the network

In [3]:
slice.get_node(name="c1").execute("ping 30.0.2.2 -c 5", quiet=False)

PING 30.0.2.2 (30.0.2.2) 56(84) bytes of data.
64 bytes from 30.0.2.2: icmp_seq=1 ttl=61 time=6.72 ms
64 bytes from 30.0.2.2: icmp_seq=2 ttl=61 time=0.344 ms
64 bytes from 30.0.2.2: icmp_seq=3 ttl=61 time=0.255 ms
64 bytes from 30.0.2.2: icmp_seq=4 ttl=61 time=0.292 ms
64 bytes from 30.0.2.2: icmp_seq=5 ttl=61 time=0.339 ms

--- 30.0.2.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4074ms
rtt min/avg/max/mdev = 0.255/1.589/6.715/2.563 ms


('PING 30.0.2.2 (30.0.2.2) 56(84) bytes of data.\n64 bytes from 30.0.2.2: icmp_seq=1 ttl=61 time=6.72 ms\n64 bytes from 30.0.2.2: icmp_seq=2 ttl=61 time=0.344 ms\n64 bytes from 30.0.2.2: icmp_seq=3 ttl=61 time=0.255 ms\n64 bytes from 30.0.2.2: icmp_seq=4 ttl=61 time=0.292 ms\n64 bytes from 30.0.2.2: icmp_seq=5 ttl=61 time=0.339 ms\n\n--- 30.0.2.2 ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 4074ms\nrtt min/avg/max/mdev = 0.255/1.589/6.715/2.563 ms\n',
 '')

In [2]:
slice.get_node(name="p1").execute("ping 40.0.2.3 -c 5", quiet=False)

PING 40.0.2.3 (40.0.2.3) 56(84) bytes of data.
64 bytes from 40.0.2.3: icmp_seq=1 ttl=61 time=1.07 ms
64 bytes from 40.0.2.3: icmp_seq=2 ttl=61 time=0.349 ms
64 bytes from 40.0.2.3: icmp_seq=3 ttl=61 time=0.291 ms
64 bytes from 40.0.2.3: icmp_seq=4 ttl=61 time=0.244 ms
64 bytes from 40.0.2.3: icmp_seq=5 ttl=61 time=0.257 ms

--- 40.0.2.3 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4054ms
rtt min/avg/max/mdev = 0.244/0.441/1.065/0.314 ms


('PING 40.0.2.3 (40.0.2.3) 56(84) bytes of data.\n64 bytes from 40.0.2.3: icmp_seq=1 ttl=61 time=1.07 ms\n64 bytes from 40.0.2.3: icmp_seq=2 ttl=61 time=0.349 ms\n64 bytes from 40.0.2.3: icmp_seq=3 ttl=61 time=0.291 ms\n64 bytes from 40.0.2.3: icmp_seq=4 ttl=61 time=0.244 ms\n64 bytes from 40.0.2.3: icmp_seq=5 ttl=61 time=0.257 ms\n\n--- 40.0.2.3 ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 4054ms\nrtt min/avg/max/mdev = 0.244/0.441/1.065/0.314 ms\n',
 '')

Install Apache on producers and set data

In [58]:
for n in ['p1','p2','p3']:
    slice.get_node(name=n).execute("sudo apt-get -y -q install apache2")
    slice.get_node(name=n).execute("sudo service apache2 start")
    slice.get_node(name=n).execute("sudo rm /var/www/html/index.html")
    slice.get_node(name=n).execute("sudo git clone https://github.com/sjzhu/CSE534.git")
    slice.get_node(name=n).execute("sudo cp -r CSE534/DataFiles /var/www/html/Data")

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  apache2-bin apache2-data apache2-utils libapr1 libaprutil1
  libaprutil1-dbd-sqlite3 libaprutil1-ldap libjansson4 liblua5.2-0 ssl-cert
Suggested packages:
  apache2-doc apache2-suexec-pristine | apache2-suexec-custom www-browser
  openssl-blacklist
The following NEW packages will be installed:
  apache2 apache2-bin apache2-data apache2-utils libapr1 libaprutil1
  libaprutil1-dbd-sqlite3 libaprutil1-ldap libjansson4 liblua5.2-0 ssl-cert
0 upgraded, 11 newly installed, 0 to remove and 58 not upgraded.
Need to get 1867 kB of archives.
After this operation, 8098 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 libapr1 amd64 1.6.5-1ubuntu1 [91.4 kB]
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates/main amd64 libaprutil1 amd64 1.6.1-4ubuntu2.2 [85.1 kB]
Get:3 http://nova.clouds.arc

Run test to download data and then also clean up the created files to allow multiple runs(ran 5 times for the paper)

In [24]:
#Run C1 500KB test
slice.get_node(name="c1").execute("echo C1-P2 500KB run:", quiet=False)
slice.get_node(name="c1").execute("time wget -q 30.0.2.2/Data/500KB.bin", quiet=False)

#Run C2 500KB test
slice.get_node(name="c2").execute("echo C2-P2 500KB run:", quiet=False)
slice.get_node(name="c2").execute("time wget -q 30.0.2.2/Data/500KB.bin", quiet=False)

#Run C1 100MB test
slice.get_node(name="c1").execute("echo C1-P2 100MB run:", quiet=False)
slice.get_node(name="c1").execute("time wget -q 30.0.2.2/Data/100MB.bin", quiet=False)

#Run C2 100MB test
slice.get_node(name="c2").execute("echo C2-P2 100MB run:", quiet=False)
slice.get_node(name="c2").execute("time wget -q 30.0.2.2/Data/100MB.bin", quiet=False)

#Run C3 500KB test
slice.get_node(name="c3").execute("echo C3-P1 500KB run:", quiet=False)
slice.get_node(name="c3").execute("time wget -q 20.0.2.1/Data/500KB.bin", quiet=False)

#Run C3 100MB test
slice.get_node(name="c3").execute("echo C3-P1 100MB run:", quiet=False)
slice.get_node(name="c3").execute("time wget -q 20.0.2.1/Data/100MB.bin", quiet=False)

#Clear all of the created files
slice.get_node(name="c1").execute("rm 500KB.bin", quiet=False)
slice.get_node(name="c2").execute("rm 500KB.bin", quiet=False)
slice.get_node(name="c1").execute("rm 100MB.bin", quiet=False)
slice.get_node(name="c2").execute("rm 100MB.bin", quiet=False)
slice.get_node(name="c3").execute("rm 500KB.bin", quiet=False)
slice.get_node(name="c3").execute("rm 100MB.bin", quiet=False)

#Check to make sure the directories are clear
slice.get_node(name="c1").execute("ls", quiet=False)
slice.get_node(name="c2").execute("ls", quiet=False)
slice.get_node(name="c3").execute("ls", quiet=False)

C1-P2 500KB run:
 
real	0m0.008s
user	0m0.003s
sys	0m0.000s
 C2-P2 500KB run:
 
real	0m0.005s
user	0m0.000s
sys	0m0.003s
 C1-P2 100MB run:
 
real	0m0.177s
user	0m0.013s
sys	0m0.157s
 C2-P2 100MB run:
 
real	0m0.138s
user	0m0.008s
sys	0m0.117s
 C3-P1 500KB run:
 
real	0m0.006s
user	0m0.003s
sys	0m0.000s
 C3-P1 100MB run:
 
real	0m0.169s
user	0m0.004s
sys	0m0.155s
 

('', '')

Use this to extend the slice when needed

In [22]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 14 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

slice.update()
_ = slice.show()

ID,e4312ec7-9de1-41a0-81b6-122f552c2486
Name,IP_Test_maross6
Lease Expiration (UTC),2023-12-03 17:39:17 +0000
Lease Start (UTC),2023-11-12 21:16:46 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


Use this section to delete the slice when needed

In [ ]:
slice.delete()


In [ ]:
# slice should end up in "Dead" state
# re-run this cell until you see it in "Dead" state
slice.update()
_ = slice.show()

Used for temp work 

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()
import os
slice_name="Project_Test_" + os.getenv('NB_USER')

slice = fablib.get_slice(slice_name)

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)

nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);